## 문자 단위 RNN 실습

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기 : {}'.format(vocab_size))
print(char_vocab)

문자 집합의 크기 : 5
['!', 'a', 'e', 'l', 'p']


In [3]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [4]:
char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [5]:
index_to_char = {}

for key,value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
x_data, y_data

([1, 4, 4, 3, 2], [4, 4, 3, 2, 0])

In [7]:
#nn.RNN()은 기본적으로 3차원 텐서를 입력받음
x_data = [x_data]
y_data = [y_data]
x_data, y_data

([[1, 4, 4, 3, 2]], [[4, 4, 3, 2, 0]])

In [8]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
x_one_hot

[array([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])]

In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\dltkd\AppData\Local\Temp\ipykernel_154064\2348034151.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  X = torch.FloatTensor(x_one_hot)


In [10]:
print('훈련 데이터 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [15]:
##RNN모델 구현(fc 완전 연결층)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [16]:
net = Net(input_size, hidden_size, output_size)

In [54]:
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [55]:
outputs.view(-1).shape

torch.Size([25])

#### torch.view와 torch.reshape의 차이
- view는 contiguous 속성이 만족되지 않는 경우 일부 사용이 제한됨
- contiguous
    - 크기가 (4,3)인 b와 (3,4)에서 transpose한 a가 있다
    - 이 두 차원을 axis방향(오른쪽 방향 우선)순서로 불러온다.
    - a는 axis=0인 오른쪽 방향으로 자료가 순서대로 저장
    - transpose한 b는 axis=1인 수직방향으로 자료가 저장
    - a는 contiguous=True, b는 contiguous=False 상태
    - stride()혹은 is_contiguous()로 contiguous여부 파악 가능
- 따라서 차원 변환을 적용하려는 상태가 확실하지 않으면 reshape 권장

In [56]:
outputs.view(-1, input_size).shape, outputs.reshape(-1, input_size).shape

(torch.Size([5, 5]), torch.Size([5, 5]))

In [57]:
outputs.data, outputs.data.numpy().argmax(axis=2)

(tensor([[[0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1.],
          [0., 0., 0., 0., 1.],
          [0., 0., 0., 1., 0.],
          [0., 0., 1., 0., 0.]]]),
 array([[1, 4, 4, 3, 2]], dtype=int64))

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [63]:
X

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])

In [18]:
epochs = 100

for i in range(epochs):
    optimizer.zero_grad()
    outputs = net(X)
    # 3차원 텐서를 view를 사용해 차원 축소
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    # 기울기 계산
    loss.backward()
    # 파라미터 업데이터 
    optimizer.step()

    # 모델이 실제 어떻게 예측하는지 확인하는 코드
    # 최종 예측값인 각 time-step별 5차원 벡터에 대해서 가장 높은 값의 인덱스 선택
    # outputs: 값과 계산과정이 기록되어있음
    # outputs.data: 값만 추출
    # .numpy(): numpy배열로 변환
    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "trueY: ", y_data, "prediction str: ", result_str)

0 loss:  1.5167362689971924 prediction:  [[0 2 2 2 2]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  !eeee
1 loss:  1.1870737075805664 prediction:  [[4 4 3 3 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
2 loss:  0.8872488141059875 prediction:  [[4 4 3 3 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
3 loss:  0.638152003288269 prediction:  [[4 4 3 3 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  ppll!
4 loss:  0.4431123733520508 prediction:  [[4 4 3 2 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.30094367265701294 prediction:  [[4 4 3 2 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.20489494502544403 prediction:  [[4 4 3 2 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.1418692171573639 prediction:  [[4 4 3 2 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.09896691143512726 prediction:  [[4 4 3 2 0]] trueY:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.06895744055509567 prediction:  [[4 4 3 2 0]] trueY:

In [ ]:
with torch.no_grad():
    